## 【問題7】（アドバンス課題）PyTorchへの書き換え
Kerasで実装したモデル・パラメータと同条件で実装

In [5]:
import time
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam

### Iris（2値分類）

In [2]:
from sklearn.model_selection import train_test_split
# データセットの読み込み
dataset_path ="/Users/tamiyagt/Documents/Machine Learning/01_github/diveintocode-ml/sprint/Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True)

# 入力データはfloat、ラベルはlong（int）に変換
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
X_test = torch.from_numpy(X_test).float()

y_train = torch.from_numpy(y_train).float()
y_val = torch.from_numpy(y_val).float()
y_test = torch.from_numpy(y_test).float()

train_set = TensorDataset(X_train, y_train)
val_set = TensorDataset(X_val, y_val)
test_set = TensorDataset(X_test, y_test)

In [6]:
class Iris_bin(torch.nn.Module):
    
    def __init__(self, n_input=4, n_hidden=8, num_classes=1):
        super().__init__()
        self.l = nn.Sequential(
            nn.Linear(n_input, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, num_classes),
            nn.Sigmoid())
        
    def forward(self, x):
        output = self.l(x)
        return output

In [7]:
model = Iris_bin()
N_EPOCHS = 100
LR = 0.01
N_BATCH = 1

criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=LR)

# 学習
model.train()
train_loader = DataLoader(train_set, batch_size=N_BATCH, shuffle=True)
total_step = len(train_loader)
start = time.time()
for epoch in range(N_EPOCHS):
    for samples, labels in train_loader:

        samples = torch.autograd.Variable(samples)
        labels = torch.autograd.Variable(labels)

        optimizer.zero_grad()  # 勾配の初期化
        outputs = model(samples)  # 出力計算
        loss = criterion(outputs, labels)  # ロス計算
        loss.backward()  # 逆伝播
        optimizer.step()  # パラメータ更新
    
    if (epoch+1) % 10 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, N_EPOCHS, loss.item()))

# 推定
total, tp = 0, 0
test_loader = DataLoader(test_set, batch_size=N_BATCH, shuffle=False)
model.eval()
for sample, label in test_loader:
    samples = torch.autograd.Variable(samples)
    labels = torch.autograd.Variable(labels)

    outputs = model(samples)
    preds = torch.eq(torch.sign(labels-0.5), torch.sign(outputs-0.5))        
    total += label.shape[0]
    tp += sum(preds)

acc = int(tp/total)
end = time.time()

print('test acc: {:.4f}'.format(acc))
print('\n wall time: {:.4f}sec'.format(end-start))

Epoch [10/100], Loss: 0.1281
Epoch [20/100], Loss: 0.0040
Epoch [30/100], Loss: 0.0044
Epoch [40/100], Loss: 0.0391
Epoch [50/100], Loss: 0.0182
Epoch [60/100], Loss: 0.0193
Epoch [70/100], Loss: 0.0047
Epoch [80/100], Loss: 0.3534
Epoch [90/100], Loss: 0.0043
Epoch [100/100], Loss: 0.0010
test acc: 1.0000

 wall time: 3.7222sec


/Users/distiller/project/conda/conda-bld/pytorch_1587428061935/work/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


### Iris（多値分類）

In [8]:
# データセットの読み込み
dataset_path ="/Users/tamiyagt/Documents/Machine Learning/01_github/diveintocode-ml/sprint/Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y[y=='Iris-setosa'] = 2
y = np.array(y).astype(np.int)
X = np.array(X)
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True)

# 入力データはfloat、ラベルはlong（int）に変換
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
X_test = torch.from_numpy(X_test).float()

y_train = torch.from_numpy(y_train).long()
y_val = torch.from_numpy(y_val).long()
y_test = torch.from_numpy(y_test).long()

train_set = TensorDataset(X_train, y_train)
val_set = TensorDataset(X_val, y_val)
test_set = TensorDataset(X_test, y_test)

/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [11]:
class Iris_multi(torch.nn.Module):
    
    def __init__(self, n_input=4, n_hidden1=16, n_hidden2=8, num_classes=3):
        super().__init__()
        self.l = nn.Sequential(
            nn.Linear(n_input, n_hidden1),
            nn.ReLU(),
            nn.Linear(n_hidden1, n_hidden2),
            nn.ReLU(),
            nn.Linear(n_hidden2, num_classes))
        
    def forward(self, x):
        output = self.l(x)
        return output

In [12]:
model = Iris_multi()
N_EPOCHS = 50
LR = 0.01
N_BATCH = 10

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

# 学習
model.train()
train_loader = DataLoader(train_set, batch_size=N_BATCH, shuffle=True)
total_step = len(train_loader)
start = time.time()
for epoch in range(N_EPOCHS):
    for samples, labels in train_loader:

        samples = torch.autograd.Variable(samples)
        labels = torch.autograd.Variable(labels)

        optimizer.zero_grad()  # 勾配の初期化
        outputs = model(samples)  # 出力計算
        loss = criterion(outputs, labels)  # ロス計算
        loss.backward()  # 逆伝播
        optimizer.step()  # パラメータ更新
        
    if (epoch+1) % 10 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, N_EPOCHS, loss.item()))

# 推定
total, tp = 0, 0
test_loader = DataLoader(test_set, batch_size=N_BATCH, shuffle=False)
model.eval()
for samples, labels in test_loader:
    samples = torch.autograd.Variable(samples)
    labels = torch.autograd.Variable(labels)

    outputs = model(samples)
    preds = torch.argmax(outputs, dim=1)
    
    total += labels.shape[0]
    tp += torch.eq(preds, labels).sum().item()

acc = tp/total
end = time.time()

print('test acc: {:.4f}'.format(acc))
print('\n wall time: {:.4f}sec'.format(end-start))

Epoch [10/50], Loss: 0.2678
Epoch [20/50], Loss: 0.3310
Epoch [30/50], Loss: 0.0248
Epoch [40/50], Loss: 0.0207
Epoch [50/50], Loss: 0.0218
test acc: 0.9000

 wall time: 0.3956sec


### House Prices

In [17]:
from sklearn.preprocessing import StandardScaler
house_data = pd.read_csv('/Users/tamiyagt/Documents/machine learning/02_Kaggle/house prices/train.csv')

#GrLivArea、YearBuilt、SalePriceを抽出
train = house_data.loc[:, ['GrLivArea', 'YearBuilt', 'SalePrice']]

# DataFrameをndarrayに変換
X = np.array(train.iloc[:, :-1])
y = np.array(train.iloc[:, -1])[:,None]

# 特徴量を標準化
scaler = StandardScaler()
X = scaler.fit_transform(X)
# 目的変数を対数変換
y = np.log1p(y)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# 入力データはfloat、ラベルはlong（int）に変換
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
X_test = torch.from_numpy(X_test).float()

y_train = torch.from_numpy(y_train).float()
y_val = torch.from_numpy(y_val).float()
y_test = torch.from_numpy(y_test).float()

train_set = TensorDataset(X_train, y_train)
val_set = TensorDataset(X_val, y_val)
test_set = TensorDataset(X_test, y_test)

In [20]:
class House_Prices(torch.nn.Module):
    
    def __init__(self, n_input=2, n_hidden1=10, n_hidden2=10, num_classes=1):
        super().__init__()
        self.l = nn.Sequential(
            nn.Linear(n_input, n_hidden1),
            nn.ReLU(),
            nn.Linear(n_hidden1, n_hidden2),
            nn.ReLU(),
            nn.Linear(n_hidden2, num_classes),
            nn.ReLU())
        
    def forward(self, x):
        output = self.l(x)
        return output

In [21]:
import time

model = House_Prices()
N_EPOCHS = 50
LR = 0.01
N_BATCH = 10

criterion = torch.nn.MSELoss()
optimizer = Adam(model.parameters(), lr=LR)

# 学習
model.train()
train_loader = DataLoader(train_set, batch_size=N_BATCH, shuffle=True)
total_step = len(train_loader)
start = time.time()
for epoch in range(N_EPOCHS):
    for samples, labels in train_loader:

        samples = torch.autograd.Variable(samples)
        labels = torch.autograd.Variable(labels)

        optimizer.zero_grad()  # 勾配の初期化
        outputs = model(samples)  # 出力計算
        loss = criterion(outputs, labels)  # ロス計算
        loss.backward()  # 逆伝播
        optimizer.step()  # パラメータ更新
        
    if (epoch+1) % 10 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, N_EPOCHS, loss.item()))

# 推定
total, tp = 0, 0
test_loader = DataLoader(test_set, batch_size=N_BATCH, shuffle=False)
model.eval()
for samples, labels in test_loader:
    samples = torch.autograd.Variable(samples)
    labels = torch.autograd.Variable(labels)

    outputs = model(samples)
    
    mse = F.mse_loss(outputs, labels)
    
end = time.time()

print('test MSE: {:.4f}'.format(mse))
print('\n wall time: {:.4f}sec'.format(end-start))

Epoch [10/50], Loss: 0.0046
Epoch [20/50], Loss: 0.0206
Epoch [30/50], Loss: 0.0198
Epoch [40/50], Loss: 0.0260
Epoch [50/50], Loss: 0.0528
test MSE: 0.0217

 wall time: 3.4426sec


### MNIST

In [22]:
#《データセットをダウンロードするコード》
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

X_train = X_train[:,None,:,:]  # NCHW
X_test = X_test[:,None,:,:]  # NCHW

# trainとvalに分割
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True)

# 入力データはfloat、ラベルはlong（int）に変換
# PyTorchはonehot encoding不要
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
X_test = torch.from_numpy(X_test).float()

y_train = torch.from_numpy(y_train).long()
y_val = torch.from_numpy(y_val).long()
y_test = torch.from_numpy(y_test).long()

train_set = TensorDataset(X_train, y_train)
val_set = TensorDataset(X_val, y_val)
test_set = TensorDataset(X_test, y_test)

In [25]:
class CNN(torch.nn.Module):
    
    def __init__(self, num_classes=10):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3),
            nn.MaxPool2d((3,3), padding=1))
        self.l2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3),
            nn.MaxPool2d((3,3), padding=1))
        self.lo = nn.Sequential(
            nn.Flatten(),
            nn.Linear(144, num_classes))
        
    def forward(self, x):
        h = F.relu(self.l1(x))
        h = F.relu(self.l2(h))
        out = self.lo(h)
        return out

In [27]:
model = CNN()
N_EPOCHS = 20
LR = 0.01
N_BATCH = 10

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)


# 学習
model.train()
train_loader = DataLoader(train_set, batch_size=N_BATCH, shuffle=True)
total_step = len(train_loader)
start = time.time()
for epoch in range(N_EPOCHS):
    for samples, labels in train_loader:

        samples = torch.autograd.Variable(samples)
        labels = torch.autograd.Variable(labels)

        optimizer.zero_grad()  # 勾配の初期化
        outputs = model(samples)  # 出力計算
        loss = criterion(outputs, labels)  # ロス計算
        loss.backward()  # 逆伝播
        optimizer.step()  # パラメータ更新
        
    print('Epoch [{}/{}], Loss: {:.4f}'
          .format(epoch+1, N_EPOCHS, loss.item()))

# 推定
total, tp = 0, 0
test_loader = DataLoader(test_set, batch_size=N_BATCH, shuffle=False)
model.eval()
for samples, labels in test_loader:
    samples = torch.autograd.Variable(samples)
    labels = torch.autograd.Variable(labels)

    outputs = model(samples)
    preds = torch.argmax(outputs, dim=1)
    
    total += labels.shape[0]
    tp += torch.eq(preds, labels).sum().item()

acc = tp/total
end = time.time()

print('test acc: {:.4f}'.format(acc))
print('\n wall time: {:.4f}sec'.format(end-start))

Epoch [1/20], Loss: 0.0368
Epoch [2/20], Loss: 0.0526
Epoch [3/20], Loss: 0.0068
Epoch [4/20], Loss: 0.0613
Epoch [5/20], Loss: 0.0189
Epoch [6/20], Loss: 0.6423
Epoch [7/20], Loss: 0.0190
Epoch [8/20], Loss: 0.0037
Epoch [9/20], Loss: 0.0101
Epoch [10/20], Loss: 0.0522
Epoch [11/20], Loss: 0.1633
Epoch [12/20], Loss: 0.1919
Epoch [13/20], Loss: 0.1376
Epoch [14/20], Loss: 0.0159
Epoch [15/20], Loss: 0.0281
Epoch [16/20], Loss: 0.0461
Epoch [17/20], Loss: 0.0086
Epoch [18/20], Loss: 0.0013
Epoch [19/20], Loss: 0.0007
Epoch [20/20], Loss: 0.0023
test acc: 0.9555

 wall time: 232.8668sec
